In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("answers.csv", index_col=0)
# El salario de las personas de Mx se guarda en la columna salarymx y está en pesos.
# Si el valor de salarymx es == 0 es porque son de otro país y su salario está en la columna salaryusd (en dólares).
# Solo vamos a tomar en cuenta los datos de personas en Mx con salario en pesos.
df = df[(df['salarymx'] > 0)]
df["salarymx"].count()

1289

In [3]:
# Agrupamos en base a años de experiencia.
exp = df.groupby("experience")["salarymx"].agg(['median', 'count'])
# Solamente tomamos en cuenta los grupos en los que tenemos por lo menos 5 observaciones
exp = exp[(exp['count'] > 4)]
exp.head()

,median,count
experience,,
0,12950.0,12
1,10000.0,77
2,17750.0,96
3,25000.0,79
4,28000.0,89


In [4]:
# Importamos lo que necesitamos para generar graficas con Bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter

output_notebook()

Loading BokehJS ...

In [5]:
# Si usamos directamente el num de observaciones para el tamaño del círculo nos da valores muy altos
# así que calculamos un valor amortiguado y lo asignamos a una nueva columna 'size'.
exp['size'] = round(np.sqrt(exp['count']/2))*3
exp.head()

,median,count,size
experience,,,
0,12950.0,12,6.0
1,10000.0,77,18.0
2,17750.0,96,21.0
3,25000.0,79,18.0
4,28000.0,89,21.0


In [45]:
source = ColumnDataSource(exp)
p = figure(plot_height=400, plot_width=600)
p.circle(x='experience', y='median', source=source, size='size')
p.title.text = 'Salario medio de acuerdo a a la experiencia'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')

hover = HoverTool()
hover.tooltips=[
    ('Experiencia', '@experience años'),
    ('Observaciones', '@count'),
    ('Salario medio', '@median{$0,0}'),
]

p.add_tools(hover)

show(p)

## Comparación por género

In [7]:
# Agrupamos por experiencia y género
gender = df[(df['experience'] > 0)].groupby(["experience", "gender"])["salarymx"].agg(['median','count'])
# Descartamos grupos con menos de 5 observaciones.
gender = gender[(gender['count'] > 4)]
gender.head()

median  count
experience gender                
1          hombre  10000.0     56
           mujer   15000.0     19
2          hombre  18000.0     77
           mujer   14600.0     18
3          hombre  26000.0     61

In [8]:
# Groupby deja experience y gender como un multiindex, en lugar de como columnas.
# Así que le damos un reset_index para que los convierta a columnas normales. 
gender = gender.reset_index()
gender.head()

,experience,gender,median,count
0,1,hombre,10000.0,56
1,1,mujer,15000.0,19
2,2,hombre,18000.0,77
3,2,mujer,14600.0,18
4,3,hombre,26000.0,61


In [9]:
# Ahora que gender ya es una columna normal, podemos usarla como referencia para crear una columna de color.
gender['color'] = ['#1f77b4' if x =='hombre' else '#e617e6' for x in gender['gender']] 

# También le agregamos el size amortiguado
gender['size'] = round(np.sqrt(gender['count']/2))*3
gender.head()

,experience,gender,median,count,color,size
0,1,hombre,10000.0,56,#1f77b4,15.0
1,1,mujer,15000.0,19,#e617e6,9.0
2,2,hombre,18000.0,77,#1f77b4,18.0
3,2,mujer,14600.0,18,#e617e6,9.0
4,3,hombre,26000.0,61,#1f77b4,18.0


In [43]:
src = ColumnDataSource(gender)
p = figure(plot_height = 400, plot_width = 600)
p.circle(x='experience', y='median', source=src, size='size', color='color')

p.title.text = 'Comparativo por género'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')

hover = HoverTool()
hover.tooltips=[
    ('Género', '@gender'),    
    ('Experiencia', '@experience años'),
    ('Observaciones', '@count'),
    ('Salario medio', '@median{$0,0}'),
]

p.add_tools(hover)

show(p)


## Breakdown por Ciudad

In [39]:
df2 = pd.read_csv("answers.csv", index_col=0)
cities = df2.groupby("city")["salarymx"].agg(['count', 'median'])
cities = cities.reset_index()
cities = cities[(cities["count"]> 10)]
cities = cities[(cities["median"]> 0)]
cities = cities.sort_values(by=['median'], ascending=False)
cities.head(30)

,city,count,median
92,Monterrey,74,45000
56,Guadalajara,156,45000
187,cdmx,15,45000
24,Cdmx,42,41000
14,CDMX,483,40600
21,Cancún,12,39000
62,Hermosillo,48,39000
96,Mérida,18,38475
29,Chihuahua,11,33000
50,Estado de México,13,31750


In [41]:
cities = df2.groupby("country")["salaryusd"].agg(['count', 'median'])
cities = cities.reset_index()
cities = cities[(cities["median"]> 0)]
cities = cities[(cities["country"]!= "Otro")]
cities = cities.sort_values(by=['median'], ascending=False)
cities.head(20)

,country,count,median
7,Estados Unidos,36,10000.0
1,Canada,3,6500.0
6,España,7,3400.0
2,Chile,4,3150.0
4,Costa Rica,2,2500.0
11,Perú,5,2000.0
9,México,1346,1895.0
5,Ecuador,10,1800.0
8,Guatemala,2,1650.0
3,Colombia,20,1450.0


In [12]:
df.head()

,created,recaptcha_score,salarymx,salaryusd,extramx,extrausd,variation,english_num,english_label,age,...,ben_gym,ben_flexhours,ben_homeoffice,ben_loan,ben_healthmajor,ben_healthminor,ben_lifeins,ben_cafeteria,ben_cellphone,ben_vouchers
id,,,,,,,,,,,,,,,,,,,,,
1,2019-12-23 22:14:57.0,NaN,80000,0,60000,0,20,4,Avanzado: Puedo conversar y escribir sin probl...,24,...,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN
2,2019-12-23 22:19:42.0,NaN,28000,0,28,0,5,1,Elemental: Sé lo básico para sobrevivir (ILR 1),29,...,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-12-23 22:19:54.0,NaN,26000,0,40000,0,4,2,Limitado: Me doy a entender pero con errores d...,34,...,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-23 22:24:05.0,NaN,35000,0,18000,0,30,3,Profesional: Puedo interactuar profesionalment...,25,...,NaN,Y,Y,NaN,NaN,NaN,NaN,Y,NaN,NaN
5,2019-12-23 22:24:23.0,NaN,17000,0,219000,0,0,1,Elemental: Sé lo básico para sobrevivir (ILR 1),28,...,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Lenguajes

El dilema con los lenguajes es que cada participante puede escoger varios lenguajes (máximo 3) que utiliza. Así que no podemos tener una única variable categórica para lenguaje, sino que tenemos una variable booleana (Y/N) por cada una de las opciones de lenguaje. Si tuvieramos una única variable, simplemente podríamos hacer un groupby y listo, pero al no tenerla, tenemos que "armar" nuestro dataframe.

Vamos a generar una lista donde cada elemento es a su vez una lista con la info de cada lenguaje (nombre del lenguaje, número de observaciones y salario medio). A partir de esa lista de listas generamos un dataframe y continuamos como de costumbre.

In [48]:
# Inicializamos nuestra lista maestra
lang_list = []
# Leemos la lista de lenguajes a partir de un catálogo.
import csv
with open("lang_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        lang_key = "lang_"+row['key']
        # Creamos una lista con el nombre del lenguaje, su num. de observaciones y salario medio, y agregamos dicha lista a nuestra lista maestra.
        lang_list.append([row['name'], df[(df[lang_key]=="Y")]["salarymx"].count(), df[(df[lang_key]=="Y")]["salarymx"].median()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
lang_df = pd.DataFrame(lang_list, columns = ['lenguaje', 'count', 'salario'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
lang_df['popularidad'] = round(np.sqrt(lang_df['count']/2)*2)
lang_df.head(30)

,lenguaje,count,salario,popularidad
0,Bash,62,40500.0,11.0
1,C#,184,34500.0,19.0
2,C/C++,26,37000.0,7.0
3,COBOL,7,35000.0,4.0
4,Dart,11,20000.0,5.0
5,Delphi,8,22500.0,4.0
6,Elixir,24,35000.0,7.0
7,Ensamblador,0,NaN,0.0
8,Go,28,43500.0,7.0
9,Groovy,14,62500.0,5.0


In [16]:
from bokeh.models import LabelSet
from bokeh.models import Range1d

src = ColumnDataSource(lang_df)
p = figure()
# Ponemos los circulos invisibles pero con tamaño para que sirvan los tooltips en hover.
p.circle(source=src, y='popularidad', x='salario', line_color=None, fill_color=None, size=20)
p.yaxis.axis_label = 'Popularidad'

# La escala de popularidad tiene unidades arbitrarias así que prefiero evitar que se despliegue.
p.yaxis.major_label_text_font_size = '0pt'

p.xaxis.axis_label = 'Salario bruto mensual (MXN)'
p.xaxis.formatter = NumeralTickFormatter(format='$0 a')    
p.x_range = Range1d(10000, 70000)

labels = LabelSet(source=src, x='salario', y='popularidad', text='lenguaje', level='glyph',
              x_offset=-10, y_offset=-5, render_mode='canvas', text_font_size="9pt", text_color='#1f77b4')
p.add_layout(labels)

hover = HoverTool()
hover.tooltips=[
    ('Lenguaje', '@lenguaje'),
    ('Observaciones', '@count'),
    ('Salario medio', '@salario{$0,0}'),
]

p.add_tools(hover)

show(p)